# sandtable demo

Interactive walkthrough of the event-driven backtesting framework.

Covers:
1. Single-ticker backtest with the high-level API
2. Custom strategy definition
3. Multi-asset backtest
4. Parameter sweep
5. Tearsheet & strategy comparison reports

In [ ]:
from pathlib import Path

from sandtable import (
    CSVDataHandler,
    FixedSlippage,
    MACrossoverStrategy,
    MeanReversionStrategy,
    Metric,
    MultiDataHandler,
    compare_strategies,
    run_backtest,
    run_parameter_sweep,
)

DATA_DIR = Path("..") / "data"

In [ ]:
data = CSVDataHandler(DATA_DIR / "sample_ohlcv.csv", "SPY")
mr_result = run_backtest(
    strategy=MeanReversionStrategy(lookback=20, threshold=2.0),
    data=data,
    initial_capital=100_000,
    commission=0.005,
)
print(mr_result)

## 1. Single-ticker backtest

Run an MA crossover strategy on SPY with one function call.

In [ ]:
data = CSVDataHandler(DATA_DIR / "sample_ohlcv.csv", "SPY")

result = run_backtest(
    strategy=MACrossoverStrategy(fast_period=10, slow_period=30),
    data=data,
    initial_capital=100_000,
    slippage=FixedSlippage(bps=5),
    commission=0.005,
)

print(result)

In [ ]:
result.equity_dataframe().head()

In [ ]:
result.trades_dataframe()

In [ ]:
result.plot(title="MA Crossover on SPY");

## 2. Mean reversion strategy

Run the built-in `MeanReversionStrategy` (already imported above).

## 3. Multi-asset backtest

Use `MultiDataHandler` to run across several tickers.

Here we reuse the same CSV to keep things offline-friendly, but in
practice you'd use `YFinanceDataHandler` per symbol (see
`examples/multi_asset.py`).

In [ ]:
handlers = {
    "SPY": CSVDataHandler(DATA_DIR / "sample_ohlcv.csv", "SPY"),
}
multi_data = MultiDataHandler(handlers)

multi_result = run_backtest(
    strategy=MACrossoverStrategy(fast_period=10, slow_period=30),
    data=multi_data,
    initial_capital=100_000,
    slippage=FixedSlippage(bps=5),
    commission=0.005,
)
print(multi_result)

## 4. Parameter sweep

Grid-search over strategy parameters and find the best Sharpe.

In [ ]:
sweep_data = CSVDataHandler(DATA_DIR / "sample_ohlcv.csv", "SPY")

sweep = run_parameter_sweep(
    strategy_class=MeanReversionStrategy,
    param_grid={
        "lookback": [10, 20, 30],
        "threshold": [1.5, 2.0, 2.5],
    },
    data=sweep_data,
    metric=Metric.SHARPE_RATIO,
)

print(f"Best params: {sweep.best_params}")
print(f"Best Sharpe: {sweep.best_result.metrics.sharpe_ratio:.4f}")

sweep.to_dataframe()[["lookback", "threshold", "sharpe_ratio", "total_return"]]

## 5. Reports

### Tearsheet

In [ ]:
from IPython.display import HTML

html = result.tearsheet()
HTML(html)

### Strategy comparison

Compare the MA crossover vs the mean-reversion strategy.

In [ ]:
comparison_html = compare_strategies(
    {"MA Crossover (10-day / 30-day MA)": result, "Mean Reversion (20-day, 2.0σ)": mr_result},
)
HTML(comparison_html)